[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/neuromatch/climate-course-content/blob/main/projects/project-notebooks/Ocean_acidification_2024.ipynb) &nbsp; <a href="https://kaggle.com/kernels/welcome?src=https://raw.githubusercontent.com/neuromatch/climate-course-content/main/projects/project-notebooks/Ocean_acidification_2024.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open in Kaggle"/></a>

# Ocean Acidification


**Content creators:** C. Gabriela Mayorga Adame, Lidia Krinova

**Content reviewers:** Will Gregory, Paul Heubel, Laura Paccini, Jenna Pearson, Ohad Zivan

**Content editors:** Paul Heubel

**Production editors:** Paul Heubel, Konstantine Tsafatinos

**Our 2024 Sponsors:** CMIP, NFDI4Earth

In [ ]:
# @title Project Background

from ipywidgets import widgets
from IPython.display import YouTubeVideo
from IPython.display import IFrame
from IPython.display import display


class PlayVideo(IFrame):
    def __init__(self, id, source, page=1, width=400, height=300, **kwargs):
        self.id = id
        if source == "Bilibili":
            src = f"https://player.bilibili.com/player.html?bvid={id}&page={page}"
        elif source == "Osf":
            src = f"https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{id}/?direct%26mode=render"
        super(PlayVideo, self).__init__(src, width, height, **kwargs)


def display_videos(video_ids, W=400, H=300, fs=1):
    tab_contents = []
    for i, video_id in enumerate(video_ids):
        out = widgets.Output()
        with out:
            if video_ids[i][0] == "Youtube":
                video = YouTubeVideo(
                    id=video_ids[i][1], width=W, height=H, fs=fs, rel=0
                )
                print(f"Video available at https://youtube.com/watch?v={video.id}")
            else:
                video = PlayVideo(
                    id=video_ids[i][1],
                    source=video_ids[i][0],
                    width=W,
                    height=H,
                    fs=fs,
                    autoplay=False,
                )
                if video_ids[i][0] == "Bilibili":
                    print(
                        f"Video available at https://www.bilibili.com/video/{video.id}"
                    )
                elif video_ids[i][0] == "Osf":
                    print(f"Video available at https://osf.io/{video.id}")
            display(video)
        tab_contents.append(out)
    return tab_contents


video_ids = [('Youtube', 'NAgrB8HxMMk'), ('Bilibili', 'BV1fM4y1x7g8')]
tab_contents = display_videos(video_ids, W=730, H=410)
tabs = widgets.Tab()
tabs.children = tab_contents
for i in range(len(tab_contents)):
    tabs.set_title(i, video_ids[i][0])
display(tabs)

In [ ]:
# @title Tutorial slides
# @markdown These are the slides for the videos in all tutorials today
from IPython.display import IFrame
link_id = "n7wdy"
# or "hbx8d"
print(f"If you want to download the slides: https://osf.io/download/{link_id}/")
IFrame(src=f"https://mfr.ca-1.osf.io/render?url=https://osf.io/{link_id}/?direct%26mode=render%26action=download%26mode=render", width=854, height=480)

Human activities release CO2 into the atmosphere, which leads to atmospheric warming and climate change. A portion of this CO2 released by human activities is absorbed into the oceans, which has a direct, chemical effect on seawater, known as ocean acidification. When CO2 combines with water in the ocean it forms carbonic acid, which makes the ocean more acidic and can have negative impacts on certain marine ecosystems (e.g., reduce the ability of calcifying organisms to form their shells and skeletons). The degree of ocean acidification is often expressed in terms of the pH of seawater, which is the measure of acidity or alkalinity such that a pH below 7 is considered acidic, and a pH greater than 7 is considered alkaline, or basic. Additional background information on ocean acidification can be found [here](https://coastadapt.com.au/ocean-acidification-and-its-effects). In this project, you will explore spatial and temporal patterns of and relationships between pH, CO2, and temperature to assess changes in ocean acidification and the impact on marine ecosystems.

**In this project**, you will analyse ocean model and observational data from global databases to extract variables like pH, CO<sub>2</sub>, and temperature, and to investigate ocean acidification process in your region of interest. This project will also be an opportunity to investigate the relationships between these variables as well as their impact on the marine ecosystems.

# Project Template

<img src='../template-images/2024_OceanAcidification.svg'/>

*Note: The dashed boxes are socio-economic questions.*

# Data Exploration Notebook
## Project Setup


In [ ]:
# google colab installs

# !mamba install 

In [ ]:
# imports

import random
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pooch
import pandas as pd
import os
import tempfile
from requests.exceptions import ReadTimeout

In [ ]:
# helper functions

def pooch_load(filelocation=None,filename=None,processor=None):
    shared_location='/home/jovyan/shared/Data/Projects/Ocean_Acidification' # this is different for each day
    user_temp_cache=tempfile.gettempdir()

    if os.path.exists(os.path.join(shared_location,filename)):
        file = os.path.join(shared_location,filename)
    else:
        file = pooch.retrieve(filelocation,known_hash=None,fname=os.path.join(user_temp_cache,filename),processor=processor)

    return file

## NOAA Ocean pH and Acidity

### Global surface ocean acidification indicators from 1750 to 2100 (NCEI Accession 0259391)

This data package contains a hybrid surface ocean acidification (OA) data product that is produced based on three recent observational data products:
- Surface Ocean CO2 Atlas (SOCAT, version 2022)
- Global Ocean Data Analysis Product version 2 (GLODAPv2, version 2022)
- Coastal Ocean Data Analysis Product in North America (CODAP-NA, version 2021), and 14 Earth System Models from the sixth phase of the Coupled Model Intercomparison Project ([CMIP6](https://github.com/ClimateMatchAcademy/course-content/blob/main/tutorials/CMIP/CMIP_resource_bank.md)).

The trajectories of ten OA indicators are included in this data product:
- Fugacity of carbon dioxide
- pH on Total Scale
- Total hydrogen ion content
- Free hydrogen ion content
- Carbonate ion content
- Aragonite saturation state
- Calcite saturation state
- Revelle Factor
- Total dissolved inorganic carbon content
- Total alkalinity content

These OA trajectories are provided under preindustrial conditions, historical conditions, and future Shared Socioeconomic Pathways: SSP1-1.9, SSP1-2.6, SSP2-4.5, SSP3-7.0, and SSP5-8.5 from 1750 to 2100 on a global surface ocean grid. These OA trajectories are improved relative to previous OA data products with respect to data quantity, spatial and temporal coverage, diversity of the underlying data and model simulations, and the provided SSPs over the 21st century.

**Citation:**
Jiang, L.-Q., Dunne, J., Carter, B. R., Tjiputra, J. F., Terhaar, J., Sharp, J. D., et al. (2023). Global surface ocean acidification indicators from 1750 to 2100. Journal of Advances in Modeling Earth Systems, 15, e2022MS003563. https://doi.org/10.1029/2022MS003563

**Dataset**: https://www.ncei.noaa.gov/data/oceans/ncei/ocads/metadata/0259391.html


We can load and visualize the **surface pH** as follows:

In [ ]:
# code to retrieve and load the data
# url_SurfacepH= 'https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0206289/Surface_pH_1770_2100/Surface_pH_1770_2000.nc' $ old CMIP5 dataset
filename_SurfacepH = 'pHT_median_historical.nc'
url_ncei = 'https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0259391/nc/median/'
url_SurfacepH = f'{url_ncei}{filename_SurfacepH}'

try:
    print(f'Downloading from {url_ncei}...')
    ds_pH = xr.open_dataset(pooch_load(url_SurfacepH,filename_SurfacepH))
except ReadTimeout:
    print(f'Downloading from OSF because of a ReadTimeoutError. Please visit {url_ncei} for more datasets.')
    link_id = "ac7zg"
    download_link = f"https://osf.io/download/{link_id}/"
    ds_pH = xr.open_dataset(pooch_load(download_link,filename_SurfacepH))
    
ds_pH

For those feeling adventurous, there are [also files](https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0259391/nc/median/) of future projected changes under various scenarios (SSP1-1.9, SSP1-2.6, SSP2-4.5, SSP3-7.0, and SSP5-8.5, recall W2D1 tutorials):
*   pHT_median_ssp119.nc
*   pHT_median_ssp126.nc
*   pHT_median_ssp245.nc
*   pHT_median_ssp370.nc
*   pHT_median_ssp585.nc

To load them, replace the filename in the path/filename line above. These data were calculated from CMIP6 models. To learn more about CMIP please see our [CMIP Resource Bank](https://github.com/ClimateMatchAcademy/course-content/blob/main/tutorials/CMIP/CMIP_resource_bank.md) and the [CMIP website](https://wcrp-cmip.org/).  

For the advanced questions in red you can use **sea surface temperature** and **CO<sub>2</sub> concentration** from [NOAA Global Monitoring Laboratory](https://gml.noaa.gov/ccgg/trends/gl_data.html)) which were previously introduced in the tutorials.
We can load and visualize this variables as follows:

In [ ]:
filename_CO2= 'co2_mm_gl.csv'
url_CO2= 'https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_gl.csv'
ds_CO2 = pd.read_csv(pooch_load(url_CO2,filename_CO2),header=55)

ds_CO2

In [ ]:
# from W1D3 tutorial 6 we have Sea Surface Temprature from 1981 to the present:
# download the monthly sea surface temperature data from NOAA Physical System
# Laboratory. The data is processed using the OISST SST Climate Data Records
# from the NOAA CDR program.
# the data downloading may take 2-3 minutes to complete.

filename_sst='sst.mon.mean.nc'
url_sst = "https://osf.io/6pgc2/download/"

ds_SST = xr.open_dataset(pooch_load(url_sst,filename_sst))
ds_SST

**Hints for socio-economic question (dashed boxes):**

Use the attached image (Figure 4 in [Kroaker et al., 2013]( https://doi.org/10.1111/gcb.12179) research paper) and this [mapping tool](https://mapper.obis.org/). Search for species of interest on the mapping tool to see their spatial global distribution and changes over time. Be critical about what you see.


![effects of ocean acidification](https://onlinelibrary.wiley.com/cms/asset/e3670b99-729f-42e6-9f5a-b2a67d5702ca/gcb12179-fig-0004-m.png)

Summary of effects of acidification among key taxonomic groups. Effects are represented as either mean percent (+) increase or percent (−) decrease in a given response.

# Further Reading

- Understanding Ocean Acidification", NOAA (https://www.fisheries.noaa.gov/insight/understanding-ocean-acidification)
- "Ocean acidification and its effects", CoastAdapt (https://coastadapt.com.au/ocean-acidification-and-its-effects)
- "Scientists Pinpoint How Ocean Acidification Weakens Coral Skeletons", WHOI (https://www.whoi.edu/press-room/news-release/scientists-identify-how-ocean-acidification-weakens-coral-skeletons/)
- "Ocean acidification and reefs", Smithonian Tropical Research Institute (https://stri.si.edu/story/ocean-acidification-and-reefs)
- Ocean Acidification | Learn Science at Scitable https://www.nature.com/scitable/knowledge/library/ocean-acidification-25822734/
- Henry, J., J. Patterson, and L. Krimsky. 2020. “Ocean Acidification: Calcifying Marine Organisms: FA220, 3/2020”. EDIS 2020 (2):4. https://doi.org/10.32473/edis-fa220-2020.
- Lauderdale, J. M., S. Dutkiewicz, R. G. Williams, and M. J. Follows (2016), Quantifying the drivers of ocean-atmosphere CO2 fluxes, Global Biogeochem. Cycles, 30  https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2016GB005400
- Increasingly Acidic Oceans Are Causing Fish to Behave Badly | Scientific American https://www.scientificamerican.com/article/increasingly-acidic-oceans-are-causing-fish-to-behave-badly/



# Other Resources

This tutorial uses data from the simulations conducted as part of the [CMIP6](https://wcrp-cmip.org/) multi-model ensemble.

For examples on how to access and analyze data, please visit the [Pangeo Cloud CMIP6 Gallery](https://gallery.pangeo.io/repos/pangeo-gallery/cmip6/index.html)

For more information on what CMIP is and how to access the data, please see this [page](https://github.com/ClimateMatchAcademy/course-content/blob/main/tutorials/CMIP/CMIP_resource_bank.md).